<a href="https://colab.research.google.com/github/Goro-python/my_NLP/blob/master/SeqtoSeq(num%20_addition).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [300]:
# coding "a+b = c"through seq to seq modelling 

In [301]:
# generate the data 
import numpy as np 
def genTrainingData(maxDigit,sizeData): 
  X = []
  Y = []
  digits= list("123456789")
  maxLen = maxDigit+1 
  for count in range(sizeData):
    a = ""
    b = ""
    c = None 
    for i in range(maxDigit):
      a+=np.random.choice(digits)
      b+=np.random.choice(digits)
    exp = a+"+"+b
    #print(a)
    #print(b)
    c = str(int(a) + int(b))
    #print(c)
    X.append(exp[::-1])
    Y.append((" "*(maxLen-len(c))+c)[::-1])
  return [X,Y] 

In [302]:
# testcase
Xtrain,Ytrain = genTrainingData(3,50000)
XVal, YVal =    genTrainingData(3,10000)
Xtest,Ytest =   genTrainingData(3,20000)

In [303]:
# create the conversion data structure
Xtrain[:10]

['987+783',
 '215+496',
 '234+781',
 '355+638',
 '948+966',
 '994+271',
 '185+782',
 '578+532',
 '965+739',
 '213+423']

In [304]:
Ytrain[0:10]

['6711',
 '6021',
 '916 ',
 '9831',
 '8151',
 '176 ',
 '868 ',
 '0111',
 '6051',
 '636 ']

In [305]:
from collections import defaultdict
class conversionTable(object):
  def __init__(self,maxLen):
    self.digits = list("0123456789+ ")
    self.indextoChar = defaultdict(str)
    self.chartoIndex = defaultdict(int)
    self.maxLen = maxLen 
  def convertChartoIndex(self):
    count = 0 
    for ch in self.digits:
      self.chartoIndex[ch] = count
      count+=1
  def convertIndextoChar(self):
    for key,val in self.chartoIndex.items():
      self.indextoChar[val] = key

  def convertExptoArr(self,exp):
    X = np.zeros((len(exp), len(self.digits)))
    for i,ch in enumerate(exp):
      index = self.chartoIndex[ch]
      X[i,index] = 1
    return X

  def convertArrtoExp(self,Arr):  # it is going to convert the 2D array in to an expression using argmax
    exp = ""
    maxArr = np.argmax(Arr,axis = 2)
    maxArr.flatten()
    maxArr.shape
    print(maxArr)
    for index in maxArr:      
      exp+=self.indextoChar[index]
    return exp


# tetscase:
ct = conversionTable(4)
ct.convertChartoIndex();
ct.convertIndextoChar();

arr = ct.convertExptoArr("732+321")   
  

In [306]:
arr

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [307]:
# convert training data to number
def convertDatatoNumeric(ct,trainingData):
  X = []
  for data in trainingData:
    convert = list(ct.convertExptoArr(data))
    X.append(convert)
  return np.array(X)

Xtr = convertDatatoNumeric(ct,Xtrain)
Ytr = convertDatatoNumeric(ct,Ytrain)

Xv = convertDatatoNumeric(ct,XVal)
Yv = convertDatatoNumeric(ct,YVal)







In [308]:
print(Ytr[0:10])

[[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 1

In [309]:
print(Xtrain[0],Ytrain[0])

987+783 6711


In [310]:
print(Ytr[0])

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [311]:
# build the model
# Deep neural Nets # LSTM mode

from tensorflow import keras
from tensorflow.keras import layers

num_layers = 1  # Try to add more LSTM layers!
MaxLen = 7

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MaxLen, len(ct.digits))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(4))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(ct.digits), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()





Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
dense_7 (Dense)              (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [312]:
 # training 
batch_size = 32
# Train the model each generation and show predictions against the validation
# dataset.
shouldITrain = False
if shouldITrain :
  model.fit(Xtr,Ytr,batch_size=batch_size,epochs=5,validation_data = (Xv,Yv))

In [316]:
Xtst = convertDatatoNumeric(ct,Xtest)
Ytst = convertDatatoNumeric(ct,Ytest)

In [330]:
# checking Accuracy 
from sklearn.metrics import accuracy_score
y_pred = model.predict(Xtst)


In [336]:
np.argmax(y_pred[0],axis=1)

array([0, 0, 0, 0])

In [337]:
y_pred[0]

array([[0.08447406, 0.08241478, 0.0825338 , 0.08400154, 0.08324261,
        0.08321586, 0.08270966, 0.08366222, 0.08376974, 0.08286   ,
        0.08391367, 0.08320203],
       [0.08520051, 0.08176292, 0.08196943, 0.08443952, 0.08315802,
        0.0831511 , 0.08229145, 0.08386449, 0.08419681, 0.08242723,
        0.08435916, 0.08317935],
       [0.08566226, 0.08129556, 0.0815727 , 0.0847298 , 0.08308662,
        0.08312496, 0.08200157, 0.08397582, 0.08459672, 0.08205356,
        0.0846981 , 0.0832023 ],
       [0.08595607, 0.08095638, 0.0812952 , 0.08492421, 0.08302917,
        0.08312519, 0.08179447, 0.08402476, 0.08495897, 0.08174149,
        0.08495529, 0.08323877]], dtype=float32)

In [340]:
shouldITrain = True
if shouldITrain :
  model.fit(Xtr,Ytr,batch_size=batch_size,epochs=2,validation_data = (Xtst,Ytst))

Epoch 1/2
1563/1563 [==============================] - 25s 16ms/step - loss: 0.0024 - accuracy: 0.9999 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 2/2
1563/1563 [==============================] - 25s 16ms/step - loss: 0.0238 - accuracy: 0.9942 - val_loss: 0.0024 - val_accuracy: 1.0000
